In [52]:
import pandas as pd
import unicodedata

In [53]:
def format_data(filename):

    df = pd.read_csv('site_csv/' + filename)
    
    entire_sup_rent = df[
        (df['Lloguer_mitja'] == 'Lloguer mitjÃ  mensual (Euros/mes)') | 
        (df['Lloguer_mitja'] == 'Lloguer mitjà mensual (Euros/mes)')
    ]['Preu']
    
    m2_rent = df[
        (df['Lloguer_mitja'] == 'Lloguer mitjÃ  per superfÃ­cie (Euros/m2 mes)') | 
        (df['Lloguer_mitja'] == 'Lloguer mitjà per superfície (Euros/m2 mes)')
    ]['Preu']
    

    df = df.drop(df[
        (df['Lloguer_mitja'] == 'Lloguer mitjÃ  per superfÃ­cie (Euros/m2 mes)') |
        (df['Lloguer_mitja'] == 'Lloguer mitjà per superfície (Euros/m2 mes)')
    ].index)

    df = df.drop('Lloguer_mitja', axis=1)
    df = df.drop('Preu', axis=1)


    df['Preu'] = entire_sup_rent.values
    df['PreuM2'] = m2_rent.values
    df.rename(columns = {'Any':'Year'}, inplace = True)

    return df

In [61]:
files = ['2014.csv', '2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv', '2020.csv', '2021.csv']

res = pd.DataFrame()
for file in files:
    df = format_data(file)
    res = pd.concat([res, df])

res

,Year,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Preu,PreuM2
0,2014,1,1,Ciutat Vella,1,el Raval,589.55,10.76
1,2014,1,1,Ciutat Vella,2,el Barri GÃ²tic,712.79,10.58
2,2014,1,1,Ciutat Vella,3,la Barceloneta,540.71,14.4
3,2014,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",673.44,11.01
4,2014,1,2,Eixample,5,el Fort Pienc,736.09,10.42
...,...,...,...,...,...,...,...,...
506,2021,4,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,1205.3,15.2
507,2021,4,10,Sant Martí,70,el Besòs i el Maresme,700.5,10.8
508,2021,4,10,Sant Martí,71,Provençals del Poblenou,977.8,13.8
509,2021,4,10,Sant Martí,72,Sant Martí de Provençals,826.2,11.7


In [62]:
res['Preu'] = res['Preu'].fillna(0)
res['PreuM2'] = res['Preu'].fillna(0)
res.replace('--', '0', inplace=True)
res = res.astype({'Preu':'float'})
res = res.astype({'PreuM2':'float'})


tmp = res.groupby(['Year','Codi_Barri']).agg({
    'Preu': ['max'],
    'PreuM2' : ['max'],
    'Codi_Districte' : ['first'],
    'Nom_Districte' : ['first'],
    'Nom_Barri' : ['first'],
})

tmp['id'] = [i for i in range(len(tmp))]
tmp = tmp.droplevel(1, axis=1)

In [63]:
tmp = tmp.replace('Ã ','à', regex=True)
tmp = tmp.replace('Ã¯','ï', regex=True)
tmp = tmp.replace('Ã³','ï', regex=True)
tmp = tmp.replace('Ã\xad','í', regex=True)
tmp = tmp.replace('Ã²','ó', regex=True)
tmp = tmp.replace('Ã©s','és', regex=True)
tmp = tmp.replace('Ã§','ç', regex=True)

tmp.to_csv('tmp.csv')